In [ ]:
!pip install google-cloud-vision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.6/514.6 kB 9.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

In [ ]:
# Import the Vision client from Google Cloud
from google.cloud import vision
from google.colab import auth
auth.authenticate_user(project_id='label-mcdonald-test')

In [ ]:
#food-related package (requires installation, e.g., using pip install food-related-package)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.corpus import wordnet as wn

food = wn.synset('food.n.02')
food_items = list(set([w for s in food.closure(lambda s:s.hyponyms()) for w in s.lemma_names()]))

In [ ]:
food_items

['pork_tenderloin',
 'plate',
 'casaba',
 'marble_cake',
 'baking_chocolate',
 'cake',
 'plum',
 'buffalofish',
 'luncheon_meat',
 'pie_crust',
 'dahl',
 'dewberry',
 'cherrystone_clam',
 'baklava',
 'bok_choy',
 'tournedos',
 'apple_tart',
 'red_cabbage',
 'pigweed',
 'julienne',
 "dragon's_eye",
 'mush',
 'gnocchi',
 'lamb_roast',
 'soft_roe',
 'pollock',
 'raw_meat',
 'flatbrod',
 'minute_steak',
 'semi-sweet_chocolate',
 'flank_steak',
 'matzo_ball',
 'pandowdy',
 'lingonberry',
 'cocoa_plum',
 'bing_cherry',
 'Empire',
 'spring_onion',
 'quahaug',
 'native_peach',
 'greengage_plum',
 'hand_cheese',
 'Valencia_orange',
 'white_potato',
 'genoise',
 'Delicious',
 'fugu',
 'pork_sausage',
 'sockeye',
 'checkerberry',
 'partridge',
 'sirloin_tip',
 'chipolata',
 'hush_puppy',
 'fettuccine_Alfredo',
 'citron',
 'pickerel',
 'sausage_roll',
 'ackee',
 'bread-stick',
 'French_sorrel',
 'snapper',
 'picnic_ham',
 'delicatessen',
 'horseflesh',
 'jumble',
 'graham_cracker',
 'Eccles_cake',

In [ ]:
if 'french_fries' in food_items:
    print("Element exists in the list")
else:
    print("Element does not exist")


Element exists in the list


In [ ]:
# Import the Vision client from Google Cloud

def localize_objects(path):

    client = vision.ImageAnnotatorClient()

    with open(path, "rb") as image_file:
        content = image_file.read()
    image = vision.Image(content=content)

    objects = client.object_localization(image=image).localized_object_annotations

    object_names = [obj.name.lower().replace(" ","_") for obj in objects]  # Extract the names


    return object_names

    #print(f"Number of objects found: {len(objects)}")
    #for object_ in objects:
    #    print(f"\n{object_.name} (confidence: {object_.score})")
    #    print("Normalized bounding polygon vertices: ")
    #    for vertex in object_.bounding_poly.normalized_vertices:
    #        print(f" - ({vertex.x}, {vertex.y})")

In [ ]:
"""Checks if any of the detected objects are food items."""

def has_food_object(object_names):
    for name in object_names:
        if name.lower() in food_items:
            return 1  # Food object found
    return 0  # No food object found

In [ ]:
"""Analyzes a single image and returns 1 if food is detected, 0 otherwise."""
def analyze_image(image_path):
    object_names = localize_objects(image_path)
    has_food = has_food_object(object_names)
    return has_food

In [ ]:
"""Analyzes all images in a folder and returns a list of results (1 or 0)."""
def analyze_images_in_folder(folder_path):
    results = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            image_path = os.path.join(folder_path, filename)
            try:
                has_food = analyze_image(image_path)
                object_names = localize_objects(image_path)
                results.append({
                    'image': filename,
                    'object_name': object_names,
                    'food_objects': has_food
                })
            except Exception as e:
                print(f"Error processing {filename}: {e}")
                results.append(0)  # Treat error as no food detected
    return results


In [ ]:
# The path to the image file you want to analyze

folder_path = "/content/drive/MyDrive/VGG16/McDonals(downloadURL)/mcdonalds_image"
label_object_result = analyze_images_in_folder(folder_path)

In [ ]:
import pandas as pd

df = pd.DataFrame(label_object_result)
df.head()

,image,object_name,food_objects
0,ERZlJghXsAA3YuR.jpg,[french_fries],1
1,ERZWUbrX0AE5tOJ.jpg,"[hamburger, tableware, tableware, food, tablew...",1
2,ERZpNw2WsAMpMKt.jpg,[],0
3,EP0qIsaWoAIVDHc.jpg,"[tableware, tableware, hamburger, bowl, tablew...",1
4,ERK-MCgXsAE-7gW.jpg,"[tableware, food]",0


In [ ]:
df.to_csv('/content/drive/MyDrive/VGG16/McDonals(downloadURL)/mcdonald_foodObj.csv', index=False)